In [1]:
# 각 항목별 가중치를 두어, 필요한 데이터를 산정 후 재계산 
# 가중치항목: 영업이익증가율, 매출액증가율, 영업대비매출액(영업이익률/매출액), ROE, 거래량
# 이상치 제거함수(detect_outliers)를 두고 이상치 내용들은 제거함

import pandas as pd
stock = pd.read_excel("Data/2024-03-22_국내주식.xlsx")

def detect_outliers(df, columns):
    q1 = df[columns].quantile(0.25)
    q3 = df[columns].quantile(0.75)
    IQR = q3 - q1

    boundary = 1.5 * IQR

    index1 = df[df[columns] > q3 + boundary].index
    index2 = df[df[columns] < q1 - boundary].index

    df[columns] = df[columns].drop(index1)
    df[columns] = df[columns].drop(index2)

    return df

realStock = detect_outliers(stock, "ROE")
realStock = detect_outliers(realStock, "거래량")
realStock = detect_outliers(realStock, "거래대금")
realStock = detect_outliers(realStock, "PBR")
realStock = detect_outliers(realStock, "PER")

realStock['영업대비매출액'] = realStock['영업이익']/realStock['매출액']
realStock['부채비율'] = realStock['부채총계'] / realStock['자산총계']

realStock = detect_outliers(realStock, "영업대비매출액")
realStock = detect_outliers(realStock, "부채비율")

realStock['R영업이익증가율'] = realStock['영업이익증가율'].rank(method='average')
realStock['R매출액증가율'] = realStock['매출액증가율'].rank(method='average')
realStock['R영업대비매출액'] = realStock['영업대비매출액'].rank(method='average')
realStock['R부채비율'] = realStock['부채비율'].rank(method='average')
realStock['R_ROE'] = realStock['ROE'].rank(method='average')
realStock['R거래량'] = realStock['거래량'].rank(method='average')
realStock['R_ROE'] = realStock['ROE'].rank(method='average')

# 각 항목에 대한 비중을 아래와 같이 조정함
# 단기적인 초점보다는 ROE(장기비중) 투자 관점에 맞추어, 지속적인 투자가 가능하도록 계산식을 산정
realStock['score'] = realStock['R영업이익증가율'] * 0.2 + realStock['R매출액증가율'] * 0.2 + realStock['R영업대비매출액'] * 0.2 + realStock['R_ROE'] * 0.25 +   realStock['R거래량'] * 0.15
dropedStock = realStock.dropna()

# 각 근거를 바탕으로 내림차순 순위로 계산
dropedStock.sort_values('score', ascending=0)

FileNotFoundError: [Errno 2] No such file or directory: 'Data/2024-03-22_국내주식.xlsx'